<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#DecisionTree" data-toc-modified-id="DecisionTree-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>DecisionTree</a></span></li><li><span><a href="#RandomForest" data-toc-modified-id="RandomForest-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>RandomForest</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Kлассификации комментариев

## Описание проекта

**Заказчик**
Интернет-магазин «Викишоп». 

**Задача**

Обучите модель классифицировать комментарии на позитивные и негативные. Метрики качества *F1* не меньше 0.75.

**Данные**

Набор данных с разметкой о токсичности правок.

## Подготовка

In [1]:
# Загружаем бибилотеки.
import pandas as pd
import numpy as np
import re

from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import set_config
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from tqdm import tqdm

import nltk
from nltk.corpus import stopwords 

import spacy

tqdm.pandas()

#nltk.download('stopwords') # Надо установить первый раз
#nltk.download('wordnet') # Надо установить первый раз
#nltk.download('punkt') # Надо установить первый раз
#nltk.download('omw-1.4') # Надо установить первый раз
#nltk.download('averaged_perceptron_tagger') # Надо установить первый раз

In [2]:
set_config(display="text")

In [3]:
#Сохраняем путь к файлу на сервере Практикума.
server_path = '/datasets/toxic_comments.csv' 


# Сохраняем путь к файлу на компьютере.
local_path = 'datasets/toxic_comments.csv'   

try:                                                   # Пробуем найти данные по сетевому адресу,
    df = pd.read_csv(server_path, index_col=0) 
except:                                                # если не находим по сетевому, ищем по локальному.
    df = pd.read_csv(local_path, index_col=0) 

In [4]:
df.sample(10)

,text,toxic
97681,La-Ahad vs. Al-Ahad \n\nThis article lists the...,0
78552,"""\n\nPlease do not remove content from Wikiped...",0
159273,Helicia DYK Issue \n\nDYK nomination of Helici...,0
80638,"Yeah, and see WP:DICK while you're at it.",1
87273,your momma is nott hott !,1
83563,Note: the edit you are referring to was not in...,0
71176,"""\nThanks. ) You rang?/My mistakes; I mean, e...",0
149435,I reorganized this article based on other simi...,0
135290,what's the problem with you \n\nmind your own ...,1
62136,"""\nBan one side of an argument by a bullshit n...",1


In [5]:
# Проверка баланса классов
round(df['toxic'].mean(), 3)

0.102

In [6]:
# Словарь стоп-слов
stop_words = set(stopwords.words('english'))
stop_words.remove('not')

In [7]:
# Загрузка spacy
nlp = spacy.load("en_core_web_sm")

In [8]:
# Lemmatize with Spacy
def text_preprocessing_spacy(text):
  text = re.sub(re.compile('<.*?>'), '', text)
  text = re.sub('[^A-Za-z ]', ' ', text).lower()
  doc = nlp(text)
  return ' '.join([token.lemma_ for token in doc])

In [9]:
df['lemm_text'] = df['text'].progress_apply(lambda text: text_preprocessing_spacy(text))

100%|██████████| 159292/159292 [23:49<00:00, 111.44it/s]


In [10]:
df.sample(10)

,text,toxic,lemm_text
158741,Political endorsements \n\nI was asked by anot...,0,political endorsement I be ask by another e...
59956,a far-right wing wanker,0,a far right wing wanker
45168,"Good, now remove the Richie Sexton and WWE inf...",0,good now remove the richie sexton and wwe in...
90641,"Changing Images \n\nHi, why are you changing m...",0,change image hi why be you change my imag...
81729,"""\n\nSpeedy deletion of Www.wikipedia.com\n A ...",0,speedy deletion of www wikipedia com a t...
157900,"""==Celtic Language?==\n\nIt says in the articl...",0,celtic language it say in the article...
150039,"Leave this out until it's actually happened, p...",0,leave this out until it s actually happen pl...
142763,"Template: English, Scottish and British monarc...",0,template english scottish and british mona...
123105,Requested move\nGrand Master Dashi (Xiaolin Sh...,0,request move grand master dashi xiaolin show...
134476,"What nonsense Gibnews! Really, you surprise me...",0,what nonsense gibnew really you surprise I...


In [11]:
# Разделяем признаки.
features = df['lemm_text']
target = df['toxic']

In [12]:
# Готовим тренировочную и тестовую выборки.
features_train, features_test, target_train, target_test = train_test_split(
    features,
    target, 
    test_size=0.3, 
    random_state=12345,
    shuffle=True,
    stratify=target
)

## Обучение

In [13]:
# Функция для вывода значений Precision, Recall, F1-мера, AUC-ROC по обученной модели
# со следующими параметрами:
#    - значения признаков;
#    - значения целевого признака;
#    - обученная модель. 

def print_metrics(features, target, model): 
    
    predicted = model.predict(features)
    probabilities_one = model.predict_proba(features)[:, 1] 

    print(f'Precision: {precision_score(target, predicted):.3f}')
    print(f'Recall: {recall_score(target, predicted):.3f}')
    print(f'F1-мера: {f1_score(target, predicted):.3f}')
    print(f'AUC-ROC: {roc_auc_score(target, probabilities_one): .3f}')

In [14]:
# Функция threshold_predict для подбора порога, для тренировочных данных и вывода значений Precision, Recall, F1-мера,  
# вывода значений Precision, Recall, F1-мера, для со следующими параметрами:
#    - обученная модель
#    - значения признаков тренировочных данных;
#    - значения целевого признака тренировочных данных;. 

def threshold_predict(model, features_train, target_train):
    probabilities_one_train = model.predict_proba(features_train)[:, 1] 

    f1_best = 0
    threshold_best = -1

    for threshold in np.arange(0, 1.01, 0.01): # цикл со смещением порога
        predicted_train = probabilities_one_train >= threshold  # 
        f1 = f1_score(target_train, predicted_train)
        if f1 > f1_best:
            f1_best = f1
            threshold_best = threshold
            
    probabilities_one_train = model.predict_proba(features_train)[:, 1] 
    precision = precision_score(target_train, probabilities_one_train > threshold_best) 
    recall = recall_score(target_train, probabilities_one_train > threshold_best)
    f1 = f1_score(target_train, probabilities_one_train > threshold_best)  
    

    print('С подбором порога')   
    print(f'Precision на тренировочных: {precision:.3f}')
    print(f'Recall на тренировочных: {recall:.3f}')
    print(f'F1-мера на тренировочных: {f1:.3f}')
    print(f'Порог: {threshold_best}')
    return [precision, recall, f1, threshold_best]

### LogisticRegression

In [15]:
params={'model__max_iter':[1000]}

In [16]:
pipeline_lf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words, min_df=20)),
    ('model', LogisticRegression(random_state=12345, solver='liblinear'))])

In [17]:
lr_model = GridSearchCV(pipeline_lf, cv=5, n_jobs=-1, param_grid=params ,scoring='roc_auc') #Задаем параметры модели и поиска.

In [18]:
%%time
lr_model.fit(features_train, target_train) # Обучение регресcии.

CPU times: total: 4.41 s
Wall time: 20.8 s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(min_df=20,
                                                        stop_words={'a',
                                                                    'about',
                                                                    'above',
                                                                    'after',
                                                                    'again',
                                                                    'against',
                                                                    'ain',
                                                                    'all', 'am',
                                                                    'an', 'and',
                                                                    'any',
                                                                    'are',
                  

In [19]:
print_metrics(features_train, target_train, lr_model.best_estimator_) # Метрики на train.

Precision: 0.941
Recall: 0.658
F1-мера: 0.774
AUC-ROC:  0.981


In [20]:
# Подберем порог улучшим F1.
lr_list = ['LogisticRegression']
lr_list.extend(threshold_predict(lr_model.best_estimator_, features_train, target_train))

С подбором порога
Precision на тренировочных: 0.842
Recall на тренировочных: 0.800
F1-мера на тренировочных: 0.820
Порог: 0.27


###  DecisionTree

In [21]:
params={'model__max_depth': [25],
    'model__max_features': [200, 400]}

In [22]:
pipeline_dt = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words, min_df=20)),
    ('model', DecisionTreeClassifier(random_state=12345))])

In [23]:
dtc_model = GridSearchCV(pipeline_dt, cv=5, n_jobs=-1, param_grid=params ,scoring='roc_auc') #Задаем параметры модели и поиска.

In [24]:
%%time
dtc_model.fit(features_train, target_train) # Обучение дерева решений

CPU times: total: 4.08 s
Wall time: 11.7 s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(min_df=20,
                                                        stop_words={'a',
                                                                    'about',
                                                                    'above',
                                                                    'after',
                                                                    'again',
                                                                    'against',
                                                                    'ain',
                                                                    'all', 'am',
                                                                    'an', 'and',
                                                                    'any',
                                                                    'are',
                  

In [25]:
dtc_model.best_params_ # Лучшие гиперпараметры.

{'model__max_depth': 25, 'model__max_features': 400}

In [26]:
%%time
print_metrics(features_train, target_train, dtc_model.best_estimator_) # Метрики на train.

Precision: 0.950
Recall: 0.505
F1-мера: 0.659
AUC-ROC:  0.810
CPU times: total: 5.94 s
Wall time: 6.27 s


In [27]:
# Подберем порог улучшим F1.
dtc_list = ['DecisionTree']
dtc_list.extend(threshold_predict(dtc_model.best_estimator_, features_train, target_train,))

С подбором порога
Precision на тренировочных: 0.937
Recall на тренировочных: 0.511
F1-мера на тренировочных: 0.661
Порог: 0.15


### RandomForest

In [28]:
params = { # Список параметров.
    'model__n_estimators': [100, 200],
    'model__max_depth': [25],
    'model__max_features': [400]
} 

In [29]:
pipeline_rf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stop_words, min_df=40)),
    ('model', RandomForestClassifier(random_state=12345,))])

In [30]:
rfc_model = GridSearchCV(pipeline_rf, cv=5, n_jobs=-1, param_grid=params ,scoring='roc_auc') #Задаем параметры модели и поиска.

In [31]:
%%time
rfc_model.fit(features_train, target_train) # Обучение случайного леса.

CPU times: total: 1min 10s
Wall time: 3min 3s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(min_df=40,
                                                        stop_words={'a',
                                                                    'about',
                                                                    'above',
                                                                    'after',
                                                                    'again',
                                                                    'against',
                                                                    'ain',
                                                                    'all', 'am',
                                                                    'an', 'and',
                                                                    'any',
                                                                    'are',
                  

In [32]:
rfc_model.best_params_ # Лучшие гиперпараметры.

{'model__max_depth': 25,
 'model__max_features': 400,
 'model__n_estimators': 200}

In [33]:
%%time
print_metrics(features_train, target_train, rfc_model.best_estimator_) # Метрики на train.

Precision: 0.996
Recall: 0.569
F1-мера: 0.724
AUC-ROC:  0.954
CPU times: total: 9.66 s
Wall time: 11.4 s


In [34]:
# Подберем порог улучшим F1.
rfc_list = ['RandomForest']
rfc_list.extend(threshold_predict(rfc_model.best_estimator_, features_train, target_train))

С подбором порога
Precision на тренировочных: 0.792
Recall на тренировочных: 0.756
F1-мера на тренировочных: 0.774
Порог: 0.11


### Вывод

In [35]:
pd.options.display.float_format = '{:.3f}'.format
columns = [
    'model',
    'Precision',
    'Recall',
    'F1-мера',
    'Порог'
]
data = []


data.append(lr_list)
data.append(dtc_list)
data.append(rfc_list)
pd.DataFrame(data=data, columns=columns)

,model,Precision,Recall,F1-мера,Порог
0,LogisticRegression,0.842,0.800,0.820,0.270
1,DecisionTree,0.937,0.511,0.661,0.150
2,RandomForest,0.792,0.756,0.774,0.110


In [36]:
probabilities = lr_model.predict_proba(features_test)[:, 1] 
f1 = f1_score(target_test, probabilities > lr_list[4])
print(f'F1 модели на тестовой выборке: {f1:.3f}')

F1 модели на тестовой выборке: 0.786


Логистическая регрессия единственная удовлетворяет условию F1-мера на тестовой выборке больше 0.75.

## Выводы

- "Деревянные" модели сильно переобучатся. Изменения гиперпараметров ситуацию не улучшают. Подбор порога результат не сильно меняет.
- Логистическая регрессия показала себя лучше "деревянных" моделей, это самая быстрая модель. Подбор порога улучшает результат.
- При расчете tf-idf использовался параметр min_df. Позволяет убрать редкие слова и ускорить обучение, но не снизить результат.
- Логистическая регрессия удовлетворяет условию - F1-мера на тестовой выборке больше 0.75.